# Multivariate Fuzzy C-medoids method: Implementation

## Equations

### $J= \sum_{i=1}^{c} \sum_{k=1}^{n} \sum_{j=1}^{p} \left(u_{ijk} \right)^{m} d_{ijk}$ - Objective function to minimize.

### $d_{ijk} = \left(x_{jk} - y_{ij} \right)^{2}$ - euclidian distance squared.

### $q = \argmin_{1 \le i \le c} \sum_{j=1}^p \sum_{k=1}^n (u_{ijk})^m \cdot d_{ijk}$ - prototype coordinate of a given cluster in feature j.

### $ u_{ijk} =  \left[\sum_{h=1}^{c}\sum_{l=1}^{p} \left(\frac{d_{ijk}}{d_{hlk}}\right)^{(1/(m-1))}  \right]^{-1} $ - membership degree of pattern k in cluster $C_{i}$ on the feature j.

### $\delta_{ik} = \sum_{j=1}^{p} u_{ijk}$ - represents an aggregation measure for all the p features.

## Constraints:

### - $u_{ijk} \in [0, 1]$ for all i, j and k;
### - $0 < \sum_{j=1}^{p} \sum_{k=1}^{n} u_{ijk} < n$ for all i and
### - $\sum_{i=1}^{c}\sum_{j=1}^{p}u_{ijk} = 1$ for all k.

## Importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [47]:
np.random.seed(42)

## Tratamento dos dados

In [103]:
df = pd.read_csv('/workspaces/Fuzzy_Clustering/datasets/wine.csv')
df = df.rename(columns={'Wine': 'Class'})
df["Class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)
labels = df["Class"].values
df.drop("Class", axis=1, inplace=True)
df = df[["Alcohol", "Malic.acid", "Proline", "Mg", "Phenols", "OD", "Nonflavanoid.phenols"]]
dados = df.to_numpy()
scaler = StandardScaler()
dados = scaler.fit_transform(dados)

/tmp/ipykernel_40185/55125924.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)


In [104]:
df.head()

,Alcohol,Malic.acid,Proline,Mg,Phenols,OD,Nonflavanoid.phenols
0,14.23,1.71,1065,127,2.80,3.92,0.28
1,13.20,1.78,1050,100,2.65,3.40,0.26
2,13.16,2.36,1185,101,2.80,3.17,0.30
3,14.37,1.95,1480,113,3.85,3.45,0.24
4,13.24,2.59,735,118,2.80,2.93,0.39


## Método de agrupamento

In [105]:
class MFCMedoids:
    def __init__(self, c, X, m):
        self.c = c
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.m = m
        self.epsilon = 1e-10  # To prevent division by zero

    def initialize_u(self):
        return np.random.dirichlet(alpha=np.ones(self.c * self.p),
                                   size=self.n).reshape(self.n, self.c, self.p)

    def find_medoids(self, X, U):
        medoids = np.zeros((self.c, self.p))
        U_m = U ** self.m  # (n, c, p)

        # Para cada possível q (0 <= q < n), criamos um tensor de distâncias quadradas para todos os outros k e p
        # (n, n, p) -> distances_squared[k, q, j] = (X[k, j] - X[q, j]) ** 2
        distances_squared = np.abs(X[:, np.newaxis, :] - X[np.newaxis, :, :])  # city block

        for i in range(self.c):
            # Para o cluster i, obtemos U_m[:, i, :] -> shape (n, p)
            # Queremos calcular o custo de cada q ser o medoide: somatório sobre j e k de u_m[k, i, j] * d(k, q, j)
            
            # Expand u_m para fazer broadcast: (n, 1, p) para multiplicar com (n, n, p)
            u_m_expanded = U_m[:, i, :][:, np.newaxis, :]  # shape (n, 1, p)

            # Custo total para cada q: soma sobre k e j
            cost_per_q = np.sum(u_m_expanded * distances_squared, axis=(0, 2))  # shape (n,)

            best_q = np.argmin(cost_per_q)
            medoids[i] = X[best_q]

        return medoids


    def get_distances(self, X, medoids):
        return np.abs(X[:, np.newaxis, :] - medoids[np.newaxis, :, :]) # city block

    def update_u(self, D):
        D = np.maximum(D, self.epsilon)  # Avoid division by zero
        ratio = (D[:, np.newaxis, np.newaxis, :, :] / D[:, :, :, np.newaxis, np.newaxis]) ** (1 / (self.m - 1))
        return 1 / np.sum(ratio, axis=(3, 4))

    def get_objective_function(self, U, D):
        return np.sum((U ** self.m) * D)

# Clustering

In [106]:
def mfcm_run(dados, num_clusters, m=2, max_iter=1000, epsilon=1e-5):
    mfcm = MFCMedoids(c=num_clusters, X=dados, m=m)  # Create the MFCMedoids object

    U = mfcm.initialize_u()  # Initialize the membership matrix

    for _ in range(max_iter):
        medoids = mfcm.find_medoids(dados, U)
        D = mfcm.get_distances(dados, medoids)
        new_U = mfcm.update_u(D)
        
        # Check for convergence
        if np.linalg.norm(U - new_U) < epsilon:
            break
        
        U = new_U

    Delta = np.sum(U, axis=2)  # Summing over the second axis (variables j)

    return medoids, U, Delta

## Simulação de Monte Carlo

In [ ]:
def monte_carlo_simulation(dados, labels, num_clusters, num_trials):
    results = []
    for _ in range(num_trials):
        print(_)
        medoids, U, Delta = mfcm_run(dados, num_clusters)
        predicted_labels = np.argmax(Delta, axis=1)
        ari = adjusted_rand_score(labels, predicted_labels)
        if ari > 0.1:
            results.append(ari)
    mean_rand_index = np.mean(results)
    std_rand_index = np.std(results)
    return mean_rand_index, std_rand_index

In [7]:
num_clusters = 3
num_trials = 100
mean_rand_index, std_rand_index = monte_carlo_simulation(dados, labels, num_clusters, num_trials)

print(f"Mean ARI: {mean_rand_index}")
print(f"Std ARI: {std_rand_index}")

Mean ARI: 0.4371712409940284
Std ARI: 5.551115123125783e-17


In [110]:
num_clusters = 3
num_trials = 100
mean_rand_index, std_rand_index = monte_carlo_simulation(dados, labels, num_clusters, num_trials)

print(f"Mean ARI: {mean_rand_index}")
print(f"Std ARI: {std_rand_index}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Mean ARI: 0.5817096237352422
Std ARI: 0.049908718998853865


In [44]:
def medoide_global_calc(dados, u_ijk, m=2):
    n_objetos, p_variaveis = dados.shape
    
    u_ijk_m = u_ijk ** m # já eleva os graus de pertinência m
    pesos_kj = np.sum(u_ijk_m, axis=0).T 

    min_dissimilarity = np.inf
    medoid_index = -1

    for q in range(n_objetos): # testa cada ponto como candidato a medoide
        candidato_q = dados[q, :]
        distancias_abs = np.abs(dados - candidato_q) # distância dos pontos ao candidato
        dissimilaridade_total_q = np.sum(pesos_kj * distancias_abs)
        
        if dissimilaridade_total_q < min_dissimilarity: # se o candidato for melhor
            min_dissimilarity = dissimilaridade_total_q
            medoid_index = q
            
    medoide_global = dados[medoid_index, :]
    return medoide_global

def indices(dados, prototipos, u_ijk, m=2):
    parar = False
    while not parar:
        n, p = dados.shape
        c, _, _ = u_ijk.shape

        u_ijk_m = u_ijk ** m
    
        grupo_nomes = [f'Grupo_{i+1}' for i in range(c)] # grupo1, grupo2, ... para output
        variavel_nomes = [f'Var_{j+1}' for j in range(p)] # var1, var2, ... para output

        z = medoide_global_calc(dados, u_ijk, m)

        # dispersão intra-grupo (J)
        dist_intra = np.abs(dados.T.reshape(1, p, n) - prototipos.reshape(c, p, 1))
        J_ijk = u_ijk_m * dist_intra
        J = np.sum(J_ijk)
        J_por_grupo = np.sum(J_ijk, axis=(1, 2)) # Ji
        J_por_variavel = np.sum(J_ijk, axis=(0, 2)) # Jj
        J_ij = np.sum(J_ijk, axis=2)               

        # dispersão inter-grupo (B)
        dist_inter = np.abs(prototipos - z)
        B_ijk = u_ijk_m * dist_inter.reshape(c, p, 1)
        B = np.sum(B_ijk)
        B_por_variavel = np.sum(B_ijk, axis=(0, 2)) # Bj
        B_por_grupo = np.sum(B_ijk, axis=(1, 2)) # Bi
        B_ij = np.sum(B_ijk, axis=2)

        # dispersão total (T)
        dist_total = np.abs(dados - z)
        dist_total_bcast = dist_total.T.reshape(1, p, n)
        T_ijk = u_ijk_m * dist_total_bcast
        T = np.sum(T_ijk)
        if T < 8:
            parar = True
            T_por_variavel = np.sum(T_ijk, axis=(0, 2)) # Tj
            T_por_grupo = np.sum(T_ijk, axis=(1, 2)) # Ti
            T_ij = np.sum(T_ijk, axis=2)

            # índices globais por variável
            R_global = B / T
            COR_j = np.divide(B_por_variavel, T_por_variavel, out=np.zeros_like(B_por_variavel), where=T_por_variavel!=0)
            CTR_j = np.divide(B_por_variavel, B, out=np.zeros_like(B_por_variavel), where=B!=0)
            df_indices_variavel = pd.DataFrame({'COR(j)': COR_j, 'CTR(j)': CTR_j}, index=variavel_nomes)

            # índices de contribuição relativa por grupo
            T_i_rel = np.divide(T_por_grupo, T, out=np.zeros_like(T_por_grupo), where=T!=0)
            J_i_rel = np.divide(J_por_grupo, J, out=np.zeros_like(J_por_grupo), where=J!=0)
            B_i_rel = np.divide(B_por_grupo, B, out=np.zeros_like(B_por_grupo), where=B!=0)
            df_indices_grupo = pd.DataFrame({'T(i)': T_i_rel, 'J(i)': J_i_rel, 'B(i)': B_i_rel}, index=grupo_nomes)

            # índices por grupo e variável
            COR_ij = np.divide(B_ij, T_por_variavel, out=np.zeros_like(B_ij), where=T_por_variavel!=0)
            df_cor_ij = pd.DataFrame(COR_ij.T, index=variavel_nomes, columns=grupo_nomes)
            
            CTR_ij = np.divide(B_ij, B_por_grupo.reshape(-1, 1), out=np.zeros_like(B_ij), where=B_por_grupo.reshape(-1, 1)!=0)
            df_ctr_ij = pd.DataFrame(CTR_ij.T, index=variavel_nomes, columns=grupo_nomes)
            
            CE_ij = np.divide(B_ij, B, out=np.zeros_like(B_ij), where=B!=0)
            df_ce_ij = pd.DataFrame(CE_ij.T, index=variavel_nomes, columns=grupo_nomes)

            resultados = {
                'T_total_bruto': T,
                'B_total_bruto': B,
                'J_total_bruto': J,
                'dispersao_bruta_por_grupo': pd.DataFrame({'Ti': T_por_grupo, 'Bi': B_por_grupo, 'Ji': J_por_grupo}, index=grupo_nomes),
                'dispersao_bruta_por_variavel': pd.DataFrame({'Tj': T_por_variavel, 'Bj': B_por_variavel, 'Jj': J_por_variavel}, index=variavel_nomes),
                'T_ij_bruto': pd.DataFrame(T_ij.T, index=variavel_nomes, columns=grupo_nomes),
                'B_ij_bruto': pd.DataFrame(B_ij.T, index=variavel_nomes, columns=grupo_nomes),
                'J_ij_bruto': pd.DataFrame(J_ij.T, index=variavel_nomes, columns=grupo_nomes),
                'R_global': R_global,
                'indices_por_variavel': df_indices_variavel,
                'indices_por_grupo': df_indices_grupo,
                'COR(i,j)': df_cor_ij,
                'CTR(i,j)': df_ctr_ij,
                'CE(i,j)': df_ce_ij
            }
            
            return resultados


medoides_resultado, U_resultado, _ = mfcm_run(dados, num_clusters=3, m=2)
u_ijk_para_analise = U_resultado.transpose(1, 2, 0)

indices = indices(
    dados=dados, 
    prototipos=medoides_resultado, 
    u_ijk=u_ijk_para_analise, 
    m=2
)

print("--- 1. Medidas de Dispersão Bruta Totais ---")
print(f"  T (Dispersão Total): {indices['T_total_bruto']:.4f}")
print(f"  B (Dispersão Inter-grupo): {indices['B_total_bruto']:.4f}")
print(f"  J (Dispersão Intra-grupo): {indices['J_total_bruto']:.4f}\n")

print("--- 2. Medidas de Dispersão Bruta por Grupo (Ti, Bi, Ji) ---")
print(indices['dispersao_bruta_por_grupo'])

print("\n--- 3. Medidas de Dispersão Bruta por Variável (Tj, Bj, Jj) ---")
print(indices['dispersao_bruta_por_variavel'])

print("\n--- 4. Medida de Dispersão Bruta T_ij ---")
print(indices['T_ij_bruto'])

print("\n--- 5. Medida de Dispersão Bruta B_ij ---")
print(indices['B_ij_bruto'])

print("\n--- 6. Medida de Dispersão Bruta J_ij ---")
print(indices['J_ij_bruto'])

print("--- 7. Índices Globais ---")
print(f"  R (Heterogeneidade Global): {indices['R_global']:.4f}\n")

print("--- 8. Índices por Variável (CORj, CTRj) ---")
print(indices['indices_por_variavel'])

print("\n--- 9. Índices de Contribuição Relativa por Grupo (T(i), B(i), J(i)) ---")
print(indices['indices_por_grupo'])

print("\n--- 10. Índice COR(i,j) ---")
print(indices['COR(i,j)'])

print("\n--- 11. Índice CTR(i,j) ---")
print(indices['CTR(i,j)'])

print("\n--- 12. Índice CE(i,j) ---")
print(indices['CE(i,j)'])

--- 1. Medidas de Dispersão Bruta Totais ---
  T (Dispersão Total): 5.8255
  B (Dispersão Inter-grupo): 11.0900
  J (Dispersão Intra-grupo): 12.5750

--- 2. Medidas de Dispersão Bruta por Grupo (Ti, Bi, Ji) ---
               Ti        Bi       Ji
Grupo_1  1.941836  3.696672  4.19168
Grupo_2  1.941836  3.696672  4.19168
Grupo_3  1.941836  3.696672  4.19168

--- 3. Medidas de Dispersão Bruta por Variável (Tj, Bj, Jj) ---
              Tj        Bj        Jj
Var_1   0.615704  2.359151  2.037249
Var_2   0.412545  2.633517  2.793576
Var_3   0.452916  0.141926  0.491242
Var_4   0.517751  0.040726  0.519096
Var_5   0.539150  1.017588  0.757618
Var_6   0.324093  0.932865  0.843119
Var_7   0.331331  0.411425  0.580069
Var_8   0.300788  0.563543  0.521969
Var_9   0.684988  0.375250  1.012023
Var_10  0.476487  0.334150  0.384472
Var_11  0.321713  0.353069  0.535884
Var_12  0.448856  0.774461  1.054439
Var_13  0.399187  1.152347  1.044281

--- 4. Medida de Dispersão Bruta T_ij ---
         Grupo_